In [535]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [536]:
data = pd.read_csv('ipl_data.csv')

In [537]:
df = pd.read_csv('IPL Matches 2008-2020.csv')

## Preprocessing

In [538]:
df.replace({'Rising Pune Supergiant': 'Rising Pune Supergiants','Delhi Daredevils':'Delhi Capitals', 'Deccan Chargers':'Sunrisers Hyderabad', 'Pune Warriors':'Rising Pune Supergiants'},inplace=True)
df

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,1216547,Dubai,2020-09-28,AB de Villiers,Dubai International Cricket Stadium,0,Royal Challengers Bangalore,Mumbai Indians,Mumbai Indians,field,Royal Challengers Bangalore,tie,NaN,Y,NaN,Nitin Menon,PR Reiffel
812,1237177,Dubai,2020-11-05,JJ Bumrah,Dubai International Cricket Stadium,0,Mumbai Indians,Delhi Capitals,Delhi Capitals,field,Mumbai Indians,runs,57.0,N,NaN,CB Gaffaney,Nitin Menon
813,1237178,Abu Dhabi,2020-11-06,KS Williamson,Sheikh Zayed Stadium,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,field,Sunrisers Hyderabad,wickets,6.0,N,NaN,PR Reiffel,S Ravi
814,1237180,Abu Dhabi,2020-11-08,MP Stoinis,Sheikh Zayed Stadium,0,Delhi Capitals,Sunrisers Hyderabad,Delhi Capitals,bat,Delhi Capitals,runs,17.0,N,NaN,PR Reiffel,S Ravi


In [539]:
df["city"].fillna("Dubai", inplace = True)

# Batsman KPIs

1. Strike rate
2. Percentage of not out
3. Percentage of dot balls
4. Percentage of boundaries
5. Man of the match count
6. Percentage of 30+ runs scored in matches

In [540]:
no_of_runs=pd.DataFrame(data.groupby('batsman')['batsman_runs'].agg(sum)).reset_index().sort_values(by='batsman_runs',ascending=False).reset_index(drop=True)
no_of_runs

,batsman,batsman_runs
0,V Kohli,5878
1,SK Raina,5368
2,DA Warner,5254
3,RG Sharma,5230
4,S Dhawan,5197
...,...,...
532,IC Pandey,0
533,S Kaushik,0
534,ND Doshi,0
535,V Pratap Singh,0


In [541]:
no_of_balls=data.groupby('batsman')['ball'].count().reset_index().sort_values(by='ball',ascending=True).reset_index(drop=True)
no_of_balls

,batsman,ball
0,C Nanda,1
1,S Lamichhane,1
2,S Kaushik,1
3,Y Prithvi Raj,1
4,YA Abdulla,1
...,...,...
532,DA Warner,3819
533,SK Raina,4041
534,RG Sharma,4088
535,S Dhawan,4208


In [542]:
boundries= data.loc[(data['batsman_runs'] == 4) | (data['batsman_runs'] == 6)]
no_of_balls_hitted_boundry=boundries.groupby('batsman')['ball'].count().reset_index().sort_values(by='ball',ascending=True).reset_index(drop=True)
no_of_balls_hitted_boundry.columns = ['batsman','boundaries']
no_of_balls_hitted_boundry

,batsman,boundaries
0,K Upadhyay,1
1,GD McGrath,1
2,KL Nagarkoti,1
3,AR Bawne,1
4,TA Boult,1
...,...,...
434,SK Raina,687
435,S Dhawan,700
436,DA Warner,705
437,V Kohli,706


In [543]:
wickets=data[data['is_wicket']==1]
batsman_out=wickets.groupby('batsman')['is_wicket'].count().reset_index().sort_values(by='is_wicket',ascending=True).reset_index(drop=True)
batsman_out.columns = ['batsman','dismissed']
batsman_out

,batsman,dismissed
0,M Prasidh Krishna,1
1,Iqbal Abdulla,1
2,BJ Haddin,1
3,BE Hendricks,1
4,SS Cottrell,1
...,...,...
497,S Dhawan,152
498,SK Raina,160
499,V Kohli,163
500,RV Uthappa,169


In [544]:
dot_balls=data[(data['batsman_runs']==0)].groupby('batsman')['ball'].count().reset_index()
dot_balls.columns = ['batsman','dot_balls']
dot_balls

,batsman,dot_balls
0,A Ashish Reddy,61
1,A Chandila,3
2,A Chopra,45
3,A Choudhary,4
4,A Dananjaya,2
...,...,...
525,YV Takawale,100
526,Yashpal Singh,39
527,Younis Khan,4
528,Yuvraj Singh,976


In [545]:
no_man_of_the_match=df.groupby('player_of_match')['id'].count().reset_index().sort_values(by='id',ascending=False).reset_index(drop=True)
no_man_of_the_match.columns = ['batsman','man_of_match']
no_man_of_the_match

,batsman,man_of_match
0,AB de Villiers,23
1,CH Gayle,22
2,RG Sharma,18
3,DA Warner,17
4,MS Dhoni,17
...,...,...
228,KK Cooper,1
229,KK Ahmed,1
230,K Rabada,1
231,JJ Roy,1


In [546]:
total_score_in_every_match=data.groupby(['batsman','id'])['batsman_runs'].sum().reset_index()
score_above_thirty=total_score_in_every_match[total_score_in_every_match['batsman_runs']>=30].reset_index()
total_matches_above_thirty=score_above_thirty.groupby('batsman')['id'].count().reset_index()
total_matches_above_thirty.columns = ['batsman','match_30+']
total_matches_above_thirty

,batsman,match_30+
0,A Ashish Reddy,2
1,A Mishra,2
2,A Symonds,14
3,AA Jhunjhunwala,2
4,AB Agarkar,1
...,...,...
239,Y Venugopal Rao,11
240,YBK Jaiswal,1
241,YK Pathan,38
242,YV Takawale,2


In [547]:
total_matches_played=total_score_in_every_match.groupby('batsman')['id'].count().reset_index()
total_matches_played.columns = ['batsman','total_matches']
total_matches_played

,batsman,total_matches
0,A Ashish Reddy,23
1,A Chandila,2
2,A Chopra,6
3,A Choudhary,3
4,A Dananjaya,1
...,...,...
532,YV Takawale,10
533,Yashpal Singh,4
534,Younis Khan,1
535,Yuvraj Singh,126


In [548]:
batsmanDf = pd.merge(no_of_runs,no_of_balls,on='batsman')
batsmanDf = batsmanDf.merge(no_of_balls_hitted_boundry,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(batsman_out,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(dot_balls,left_on='batsman',right_on='batsman',how='left')
batsmanDf = batsmanDf.merge(no_man_of_the_match, left_on='batsman', right_on='batsman', how='left')
batsmanDf = batsmanDf.merge(total_matches_above_thirty, left_on='batsman', right_on='batsman', how='left')
batsmanDf = batsmanDf.merge(total_matches_played, left_on='batsman', right_on='batsman', how='left')

#filtered based on atleast 100 balls played
batsmanDf = batsmanDf[batsmanDf["ball"] >= 100]

batsmanDf

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175
...,...,...,...,...,...,...,...,...,...
243,CJ Ferguson,98,119,9.0,6.0,58.0,NaN,NaN,8
246,DJ Jacobs,92,101,14.0,7.0,62.0,NaN,1.0,7
250,RT Ponting,91,129,7.0,8.0,69.0,NaN,NaN,9
253,SL Malinga,88,103,11.0,12.0,59.0,6.0,NaN,20


In [549]:
batsmanDf['strike_rate']=np.round(batsmanDf['batsman_runs']/batsmanDf['ball']*100,2)
batsmanDf['%age_not_out']=np.round(100-(batsmanDf['dismissed']/batsmanDf['ball'])*100,2)
batsmanDf['%age_dot_balls']=np.round((batsmanDf['dot_balls']/batsmanDf['ball'])*100,2)
batsmanDf['%age_boundaries']=np.round((batsmanDf['boundaries']/batsmanDf['ball'])*100,2)
batsmanDf['%30+runs']=np.round((batsmanDf['match_30+']/batsmanDf['total_matches'])*100,2)
batsmanDf

,batsman,batsman_runs,ball,boundaries,dismissed,dot_balls,man_of_match,match_30+,total_matches,strike_rate,%age_not_out,%age_dot_balls,%age_boundaries,%30+runs
0,V Kohli,5878,4609,706.0,163.0,1625.0,13.0,84.0,184,127.53,96.46,35.26,15.32,45.65
1,SK Raina,5368,4041,687.0,160.0,1412.0,14.0,76.0,189,132.84,96.04,34.94,17.00,40.21
2,DA Warner,5254,3819,705.0,126.0,1429.0,17.0,76.0,142,137.58,96.70,37.42,18.46,53.52
3,RG Sharma,5230,4088,672.0,177.0,1532.0,18.0,73.0,194,127.94,95.67,37.48,16.44,37.63
4,S Dhawan,5197,4208,700.0,152.0,1613.0,8.0,72.0,175,123.50,96.39,38.33,16.63,41.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,CJ Ferguson,98,119,9.0,6.0,58.0,NaN,NaN,8,82.35,94.96,48.74,7.56,NaN
246,DJ Jacobs,92,101,14.0,7.0,62.0,NaN,1.0,7,91.09,93.07,61.39,13.86,14.29
250,RT Ponting,91,129,7.0,8.0,69.0,NaN,NaN,9,70.54,93.80,53.49,5.43,NaN
253,SL Malinga,88,103,11.0,12.0,59.0,6.0,NaN,20,85.44,88.35,57.28,10.68,NaN


# Bowler KPIs

1. Economy
2. Strike Rate
3. Average Wickets per match
4. Percentage of Dot balls
5. Percentage Boundaries
6. Economy for Powerplay, Middle over, Death overs
7. Bowling Average
8. 4 Wicket Haul

In [550]:
Bowler_over= data[data["extra_runs"]==0][["ball","bowler"]]
Bowler_over= Bowler_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_over["No_of_Overs"]=(np.floor(Bowler_over["ball"]/6))
Bowler_over

,bowler,ball,No_of_Overs
0,Harbhajan Singh,3325,554.0
1,PP Chawla,3218,536.0
2,R Ashwin,3186,531.0
3,A Mishra,3128,521.0
4,SL Malinga,2753,458.0
...,...,...,...
415,LS Livingstone,6,1.0
416,SN Khan,2,0.0
417,SPD Smith,2,0.0
418,DA Warner,1,0.0


In [551]:
Bowler_Runs= data.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_Runs.columns=["bowler","Total_runs_given"]
Bowler_over = Bowler_over[Bowler_over["ball"] >= 60]
Bowler_Runs

,bowler,Total_runs_given
0,PP Chawla,4330
1,Harbhajan Singh,4038
2,A Mishra,3913
3,DJ Bravo,3869
4,R Ashwin,3756
...,...,...
415,SN Khan,6
416,AM Rahane,5
417,SPD Smith,5
418,DA Warner,2


In [552]:
Bowler_boundary = data.query('batsman_runs == 4 | batsman_runs == 6')
Bowler_boundary = Bowler_boundary.groupby("bowler")["batsman_runs"].count().reset_index().sort_values(by="batsman_runs",ascending=False).reset_index(drop=True)
Bowler_boundary.columns=["bowler","Total_boundaries_given"]
Bowler_boundary

,bowler,Total_boundaries_given
0,UT Yadav,509
1,PP Chawla,497
2,DJ Bravo,443
3,P Kumar,442
4,Harbhajan Singh,431
...,...,...
411,Y Gnaneswara Rao,1
412,SPD Smith,1
413,SN Khan,1
414,SA Yadav,1


In [553]:
Bowler_wicket = data[data["dismissal_kind"] != "run out"]
Bowler_wicket= Bowler_wicket.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_wicket.columns=["bowler","Wickets"]
Bowler_wicket

,bowler,Wickets
0,SL Malinga,170
1,A Mishra,160
2,PP Chawla,156
3,DJ Bravo,153
4,Harbhajan Singh,150
...,...,...
415,BJ Rohrer,0
416,P Ray Barman,0
417,P Prasanth,0
418,P Dubey,0


In [554]:
Bowler_Dots = data[data["total_runs"] == 0]
Bowler_Dots= Bowler_Dots.groupby("bowler")["total_runs"].count().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_Dots.columns=["bowler","Dot_balls"]
Bowler_Dots

,bowler,Dot_balls
0,Harbhajan Singh,1244
1,R Ashwin,1166
2,B Kumar,1155
3,SL Malinga,1144
4,PP Chawla,1137
...,...,...
404,C Ganapathy,1
405,BJ Rohrer,1
406,B Chipli,1
407,FY Fazal,1


In [555]:
Bowler_matches = data.groupby("bowler")["id"].nunique().reset_index().sort_values(by="id",ascending=False).reset_index(drop=True)
Bowler_matches.columns=["bowler","no_of_match"]
Bowler_matches

,bowler,no_of_match
0,PP Chawla,163
1,Harbhajan Singh,157
2,RA Jadeja,155
3,R Ashwin,151
4,A Mishra,150
...,...,...
415,LA Carseldine,1
416,AA Noffke,1
417,AA Kazi,1
418,C Ganapathy,1


In [556]:
Bowler_powerplay_wickets = data[data["over"] < 6]
Bowler_powerplay_wickets =  Bowler_powerplay_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_powerplay_wickets.columns=["bowler","Powerplay_runs"]

Bowler_powerplay_runs = data.query('dismissal_kind != "run out" & over < 6')
Bowler_powerplay_runs= Bowler_powerplay_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_powerplay_runs.columns=["bowler","Powerplay_wickets"]

Bowler_powerplay_over = data.query('extra_runs == 0 & over < 6')
Bowler_powerplay_over= Bowler_powerplay_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_powerplay_over["Powerplay_over"]=np.round(Bowler_powerplay_over["ball"]/6,1)

Bowler_powerplay = pd.merge(Bowler_powerplay_runs,Bowler_powerplay_wickets,on="bowler")
Bowler_powerplay = pd.merge(Bowler_powerplay, Bowler_powerplay_over, on="bowler")
Bowler_powerplay["Powerplay_economy"] = np.round(Bowler_powerplay["Powerplay_runs"] / Bowler_powerplay["Powerplay_over"],1)

del Bowler_powerplay['ball']
Bowler_powerplay

,bowler,Powerplay_wickets,Powerplay_runs,Powerplay_over,Powerplay_economy
0,Z Khan,53,1492,209.7,7.1
1,Sandeep Sharma,53,1447,201.2,7.2
2,B Kumar,48,1472,234.0,6.3
3,UT Yadav,45,1424,174.3,8.2
4,DS Kulkarni,44,1274,165.8,7.7
...,...,...,...,...,...
339,CJ Green,0,16,2.0,8.0
340,MM Ali,0,31,4.8,6.5
341,MJ Henry,0,36,3.0,12.0
342,MG Neser,0,18,1.0,18.0


In [557]:
Bowler_middle_wickets = data.query('over >= 6 & over <= 15')
Bowler_middle_wickets =  Bowler_middle_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_middle_wickets.columns=["bowler","Middle_runs"]

Bowler_middle_runs = data.query('dismissal_kind != "run out" & over >= 6 & over <= 15')
Bowler_middle_runs= Bowler_middle_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_middle_runs.columns=["bowler","Middle_wickets"]

Bowler_middle_over = data.query('extra_runs == 0 & over >= 6 & over <= 15')
Bowler_middle_over= Bowler_middle_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_middle_over["Middle_over"]=np.round(Bowler_middle_over["ball"]/6,1)

Bowler_middle = pd.merge(Bowler_middle_runs,Bowler_middle_wickets,on="bowler")
Bowler_middle = pd.merge(Bowler_middle, Bowler_middle_over,on="bowler")
Bowler_middle["Middle_economy"] = np.round(Bowler_middle["Middle_runs"] / Bowler_middle["Middle_over"],1)

del Bowler_middle['ball']
Bowler_middle

,bowler,Middle_wickets,Middle_runs,Middle_over,Middle_economy
0,A Mishra,130,3319,457.3,7.3
1,PP Chawla,112,3463,436.2,7.9
2,Harbhajan Singh,101,2904,404.0,7.2
3,YS Chahal,94,2065,271.2,7.6
4,RA Jadeja,84,2854,371.3,7.7
...,...,...,...,...,...
401,BJ Rohrer,0,12,1.0,12.0
402,SC Kuggeleijn,0,27,3.0,9.0
403,MG Neser,0,11,1.0,11.0
404,MJ Henry,0,27,1.0,27.0


In [558]:
Bowler_death_wickets = data.query('over >= 16')
Bowler_death_wickets =  Bowler_death_wickets.groupby("bowler")["total_runs"].sum().reset_index().sort_values(by="total_runs",ascending=False).reset_index(drop=True)
Bowler_death_wickets.columns=["bowler","Death_wickets"]

Bowler_death_runs = data.query('dismissal_kind != "run out" & over >= 16')
Bowler_death_runs= Bowler_death_runs.groupby("bowler")["player_dismissed"].count().reset_index().sort_values(by="player_dismissed",ascending=False).reset_index(drop=True)
Bowler_death_runs.columns=["bowler","Death_runs"]

Bowler_death_over = data.query('extra_runs == 0 & over >= 16')
Bowler_death_over= Bowler_death_over.groupby("bowler")["ball"].count().reset_index().sort_values(by="ball",ascending=False).reset_index(drop=True)
Bowler_death_over["Death_over"]=np.round(Bowler_death_over["ball"]/6,1)

Bowler_death = pd.merge(Bowler_death_wickets,Bowler_death_runs,on="bowler")
Bowler_death = pd.merge(Bowler_death, Bowler_death_over,on="bowler")
Bowler_death["Death_economy"] = np.round(Bowler_death["Death_runs"] / Bowler_death["Death_over"],1)

del Bowler_death['ball']
Bowler_death

,bowler,Death_wickets,Death_runs,Death_over,Death_economy
0,DJ Bravo,1599,82,155.2,0.5
1,B Kumar,1267,66,129.2,0.5
2,SL Malinga,1252,90,147.0,0.6
3,JJ Bumrah,1208,53,127.5,0.4
4,UT Yadav,1072,42,95.7,0.4
...,...,...,...,...,...
331,MJ Santner,1,1,1.0,1.0
332,DJ Muthuswami,1,1,1.0,1.0
333,JM Kemp,1,0,0.2,0.0
334,AJ Finch,1,0,0.2,0.0


In [559]:
data_without_runouts = data[data["dismissal_kind"] != "run out"]
no_of_4_wickets=data_without_runouts.groupby(['bowler','id'])['is_wicket'].agg(sum).reset_index(name="wickets")
no_of_4_wickets

,bowler,id,wickets
0,A Ashish Reddy,548329,0
1,A Ashish Reddy,548341,2
2,A Ashish Reddy,548346,1
3,A Ashish Reddy,548348,1
4,A Ashish Reddy,548352,1
...,...,...,...
9661,Z Khan,1082622,0
9662,Z Khan,1082635,0
9663,Z Khan,1082640,0
9664,Z Khan,1082642,2


In [560]:
no_of_4_wickets=no_of_4_wickets[no_of_4_wickets['wickets']>=4]
no_of_4_wickets=no_of_4_wickets.groupby('bowler').size().reset_index(name='4W')
no_of_4_wickets

,bowler,4W
0,A Chandila,1
1,A Kumble,3
2,A Mishra,4
3,A Nehra,1
4,A Singh,1
...,...,...
90,VY Mahesh,1
91,YA Abdulla,2
92,YS Chahal,2
93,Yuvraj Singh,2


In [561]:
Bowler_details = Bowler_over.merge(Bowler_Runs,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_wicket,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_matches,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_boundary,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_Dots,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_powerplay,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_middle,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(Bowler_death,left_on='bowler',right_on='bowler',how='left')
Bowler_details = Bowler_details.merge(no_of_4_wickets,left_on='bowler',right_on='bowler',how='left')
Bowler_details

,bowler,ball,No_of_Overs,Total_runs_given,Wickets,no_of_match,Total_boundaries_given,Dot_balls,Powerplay_wickets,Powerplay_runs,...,Powerplay_economy,Middle_wickets,Middle_runs,Middle_over,Middle_economy,Death_wickets,Death_runs,Death_over,Death_economy,4W
0,Harbhajan Singh,3325,554.0,4038,150,157,431,1244,30.0,894.0,...,7.3,101.0,2904.0,404.0,7.2,240.0,19.0,28.2,0.7,2.0
1,PP Chawla,3218,536.0,4330,156,163,497,1137,18.0,445.0,...,7.6,112.0,3463.0,436.2,7.9,422.0,26.0,41.7,0.6,2.0
2,R Ashwin,3186,531.0,3756,138,151,361,1166,42.0,1003.0,...,6.7,83.0,2384.0,338.0,7.1,369.0,13.0,42.2,0.3,1.0
3,A Mishra,3128,521.0,3913,160,150,394,1125,6.0,164.0,...,7.6,130.0,3319.0,457.3,7.3,430.0,24.0,42.3,0.6,4.0
4,SL Malinga,2753,458.0,3486,170,122,400,1144,37.0,1210.0,...,6.7,43.0,1024.0,131.2,7.8,1252.0,90.0,147.0,0.6,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,A Flintoff,65,10.0,106,2,3,14,20,0.0,21.0,...,10.5,1.0,33.0,5.8,5.7,52.0,1.0,3.0,0.3,NaN
294,DNT Zoysa,65,10.0,100,2,3,15,24,0.0,45.0,...,7.5,1.0,21.0,2.0,10.5,34.0,1.0,2.8,0.4,NaN
295,MJ Clarke,64,10.0,72,2,5,6,22,NaN,NaN,...,NaN,2.0,72.0,10.7,6.7,NaN,NaN,NaN,NaN,NaN
296,S Dube,63,10.0,83,4,7,8,16,NaN,NaN,...,NaN,2.0,38.0,5.8,6.6,45.0,2.0,4.7,0.4,NaN


In [562]:
Bowler_details["Economy"] = np.round(Bowler_details["Total_runs_given"]/Bowler_details["No_of_Overs"],1)
Bowler_details["Strike Rate"] = np.round(Bowler_details["ball"]/Bowler_details["Wickets"],1)
Bowler_details["Avg. Wickets"] = np.round(Bowler_details["Wickets"]/Bowler_details["no_of_match"],1)
Bowler_details["%age of dots"] = np.round((Bowler_details["Dot_balls"]/Bowler_details["ball"]) * 100,1)
Bowler_details["%boundaries"] = np.round((Bowler_details["Total_boundaries_given"]/Bowler_details["ball"]) * 100,1)
Bowler_details["bowl_avg"] = np.round((Bowler_details["Total_runs_given"]/Bowler_details["Wickets"]),1)
Bowler_details.fillna(0.0)
Bowler_details.replace(np.nan, 0)
Bowler_details

,bowler,ball,No_of_Overs,Total_runs_given,Wickets,no_of_match,Total_boundaries_given,Dot_balls,Powerplay_wickets,Powerplay_runs,...,Death_runs,Death_over,Death_economy,4W,Economy,Strike Rate,Avg. Wickets,%age of dots,%boundaries,bowl_avg
0,Harbhajan Singh,3325,554.0,4038,150,157,431,1244,30.0,894.0,...,19.0,28.2,0.7,2.0,7.3,22.2,1.0,37.4,13.0,26.9
1,PP Chawla,3218,536.0,4330,156,163,497,1137,18.0,445.0,...,26.0,41.7,0.6,2.0,8.1,20.6,1.0,35.3,15.4,27.8
2,R Ashwin,3186,531.0,3756,138,151,361,1166,42.0,1003.0,...,13.0,42.2,0.3,1.0,7.1,23.1,0.9,36.6,11.3,27.2
3,A Mishra,3128,521.0,3913,160,150,394,1125,6.0,164.0,...,24.0,42.3,0.6,4.0,7.5,19.6,1.1,36.0,12.6,24.5
4,SL Malinga,2753,458.0,3486,170,122,400,1144,37.0,1210.0,...,90.0,147.0,0.6,7.0,7.6,16.2,1.4,41.6,14.5,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,A Flintoff,65,10.0,106,2,3,14,20,0.0,21.0,...,1.0,3.0,0.3,NaN,10.6,32.5,0.7,30.8,21.5,53.0
294,DNT Zoysa,65,10.0,100,2,3,15,24,0.0,45.0,...,1.0,2.8,0.4,NaN,10.0,32.5,0.7,36.9,23.1,50.0
295,MJ Clarke,64,10.0,72,2,5,6,22,NaN,NaN,...,NaN,NaN,NaN,NaN,7.2,32.0,0.4,34.4,9.4,36.0
296,S Dube,63,10.0,83,4,7,8,16,NaN,NaN,...,2.0,4.7,0.4,NaN,8.3,15.8,0.6,25.4,12.7,20.8


# Team KPIs

1. Percentage wins
2. Percentage wins by chasing
3. Percentage wins by defending
4. Percentage wins by tie
5. Percentage toss wins
6. Percentage home wins
7. Number of times qualified

In [570]:
no_of_match_as_team1=df.groupby('team1').size().reset_index(name="no_of_match_as_team1")
no_of_match_as_team1=no_of_match_as_team1.rename(columns={"team1":"team_name"})
no_of_match_as_team1

,team_name,no_of_match_as_team1
0,Chennai Super Kings,94
1,Delhi Capitals,102
2,Gujarat Lions,16
3,Kings XI Punjab,92
4,Kochi Tuskers Kerala,7
5,Kolkata Knight Riders,95
6,Mumbai Indians,97
7,Rajasthan Royals,70
8,Rising Pune Supergiants,37
9,Royal Challengers Bangalore,108


In [571]:
no_of_match_as_team2=df.groupby('team2').size().reset_index(name="no_of_match_as_team2")
no_of_match_as_team2=no_of_match_as_team2.rename(columns={"team2":"team_name"})
no_of_match_as_team2

,team_name,no_of_match_as_team2
0,Chennai Super Kings,84
1,Delhi Capitals,92
2,Gujarat Lions,14
3,Kings XI Punjab,98
4,Kochi Tuskers Kerala,7
5,Kolkata Knight Riders,97
6,Mumbai Indians,106
7,Rajasthan Royals,91
8,Rising Pune Supergiants,39
9,Royal Challengers Bangalore,87


In [572]:
no_of_matches=pd.merge(no_of_match_as_team1,no_of_match_as_team2,on="team_name")
no_of_matches['total_no_of_matches_played']=no_of_matches['no_of_match_as_team1']+no_of_matches['no_of_match_as_team2']
no_of_matches

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played
0,Chennai Super Kings,94,84,178
1,Delhi Capitals,102,92,194
2,Gujarat Lions,16,14,30
3,Kings XI Punjab,92,98,190
4,Kochi Tuskers Kerala,7,7,14
5,Kolkata Knight Riders,95,97,192
6,Mumbai Indians,97,106,203
7,Rajasthan Royals,70,91,161
8,Rising Pune Supergiants,37,39,76
9,Royal Challengers Bangalore,108,87,195


In [573]:
no_of_wins=df.groupby('winner').size().reset_index(name="no_of_wins")
no_of_wins=no_of_wins.rename(columns={'winner':'team_name'})
no_of_wins

,team_name,no_of_wins
0,Chennai Super Kings,106
1,Delhi Capitals,86
2,Gujarat Lions,13
3,Kings XI Punjab,88
4,Kochi Tuskers Kerala,6
5,Kolkata Knight Riders,99
6,Mumbai Indians,120
7,Rajasthan Royals,81
8,Rising Pune Supergiants,27
9,Royal Challengers Bangalore,91


In [574]:
teams_df=pd.merge(no_of_matches,no_of_wins,on='team_name')
teams_df

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins
0,Chennai Super Kings,94,84,178,106
1,Delhi Capitals,102,92,194,86
2,Gujarat Lions,16,14,30,13
3,Kings XI Punjab,92,98,190,88
4,Kochi Tuskers Kerala,7,7,14,6
5,Kolkata Knight Riders,95,97,192,99
6,Mumbai Indians,97,106,203,120
7,Rajasthan Royals,70,91,161,81
8,Rising Pune Supergiants,37,39,76,27
9,Royal Challengers Bangalore,108,87,195,91


In [575]:
win_by_defending=df[df['result']=='runs'].groupby('winner').size().reset_index(name="wins_defend")
win_by_defending=win_by_defending.rename(columns={'winner':'team_name'})
win_by_defending

,team_name,wins_defend
0,Chennai Super Kings,53
1,Delhi Capitals,34
2,Gujarat Lions,1
3,Kings XI Punjab,40
4,Kochi Tuskers Kerala,2
5,Kolkata Knight Riders,41
6,Mumbai Indians,62
7,Rajasthan Royals,28
8,Rising Pune Supergiants,13
9,Royal Challengers Bangalore,38


In [576]:
win_by_chase=df[df['result']=='wickets'].groupby('winner').size().reset_index(name="wins_chase")
win_by_chase=win_by_chase.rename(columns={'winner':'team_name'})
win_by_chase

,team_name,wins_chase
0,Chennai Super Kings,53
1,Delhi Capitals,50
2,Gujarat Lions,12
3,Kings XI Punjab,45
4,Kochi Tuskers Kerala,4
5,Kolkata Knight Riders,57
6,Mumbai Indians,56
7,Rajasthan Royals,51
8,Rising Pune Supergiants,14
9,Royal Challengers Bangalore,51


In [577]:
toss_wins=df.groupby('toss_winner').size().reset_index(name="toss_wins")
toss_wins=toss_wins.rename(columns={'toss_winner':'team_name'})
toss_wins

,team_name,toss_wins
0,Chennai Super Kings,97
1,Delhi Capitals,100
2,Gujarat Lions,15
3,Kings XI Punjab,85
4,Kochi Tuskers Kerala,8
5,Kolkata Knight Riders,98
6,Mumbai Indians,106
7,Rajasthan Royals,87
8,Rising Pune Supergiants,33
9,Royal Challengers Bangalore,87


In [578]:
win_by_tie=df[df['result']=='tie'].groupby('winner').size().reset_index(name="wins_tie")
win_by_tie=win_by_tie.rename(columns={'winner':'team_name'})
win_by_tie

,team_name,wins_tie
0,Delhi Capitals,2
1,Kings XI Punjab,3
2,Kolkata Knight Riders,1
3,Mumbai Indians,2
4,Rajasthan Royals,2
5,Royal Challengers Bangalore,2
6,Sunrisers Hyderabad,1


In [579]:
team_df = pd.merge(teams_df,win_by_chase,on="team_name")
team_df = pd.merge(team_df,win_by_defending,on="team_name")
team_df = team_df.merge(win_by_tie, left_on='team_name', right_on='team_name', how='left')
team_df = pd.merge(team_df, toss_wins, on ='team_name')
team_df

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97
1,Delhi Capitals,102,92,194,86,50,34,2.0,100
2,Gujarat Lions,16,14,30,13,12,1,NaN,15
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8
5,Kolkata Knight Riders,95,97,192,99,57,41,1.0,98
6,Mumbai Indians,97,106,203,120,56,62,2.0,106
7,Rajasthan Royals,70,91,161,81,51,28,2.0,87
8,Rising Pune Supergiants,37,39,76,27,14,13,NaN,33
9,Royal Challengers Bangalore,108,87,195,91,51,38,2.0,87


In [580]:
teams_df['%win']=np.round((teams_df['no_of_wins']/teams_df['total_no_of_matches_played'])*100,1)
team_df['%win_chase']=np.round((team_df['wins_chase']/team_df['no_of_wins'])*100,2)
team_df['%win_defend']=np.round((team_df['wins_defend']/team_df['no_of_wins'])*100,2)
team_df['%win_tie']=np.round((team_df['wins_tie']/team_df['no_of_wins'])*100,1)
team_df['%toss_wins']=np.round((team_df['toss_wins']/team_df['total_no_of_matches_played'])*100,1)
team_df

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1
5,Kolkata Knight Riders,95,97,192,99,57,41,1.0,98,57.58,41.41,1.0,51.0
6,Mumbai Indians,97,106,203,120,56,62,2.0,106,46.67,51.67,1.7,52.2
7,Rajasthan Royals,70,91,161,81,51,28,2.0,87,62.96,34.57,2.5,54.0
8,Rising Pune Supergiants,37,39,76,27,14,13,NaN,33,51.85,48.15,NaN,43.4
9,Royal Challengers Bangalore,108,87,195,91,51,38,2.0,87,56.04,41.76,2.2,44.6


In [581]:
data_citiy=pd.DataFrame(df.groupby('city'))
homegrounds={'Kolkata':'Kolkata Knight Riders','Bangalore':'Royal Challengers Bangalore','Bengaluru':'Royal Challengers Bangalore',
            'Chandigarh':'Kings XI Punjab','Chennai':'Chennai Super Kings','Delhi':'Delhi Capitals','Hyderabad':'Sunrisers Hyderabad',
            'Jaipur':'Rajasthan Royals','Rajkot':'Gujarat Lions','Kochi':'Kochi Tuskers Kerala','Kanpur':'Gujarat Lions',
            'Visakhapatnam':'Rising Pune Supergiants','Pune':'Rising Pune Supergiants','Cuttack':'Sunrisers Hyderabad',
            'Dharamshala':'Kings XI Punjab','Ahmedabad':'Rajasthan Royals','Mohali':'Kings XI Punjab','Mumbai':'Mumbai Indians'}

In [582]:
data_without_neutral=df[df['neutral_venue']!=1]
data_without_neutral=data_without_neutral[data_without_neutral['city']!='Dubai']
data_without_neutral['home_venue']=data_without_neutral['city'].map(homegrounds)
data_without_neutral

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,home_venue
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,Royal Challengers Bangalore
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,Kings XI Punjab
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi Capitals
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,Mumbai Indians
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,Kolkata Knight Riders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,1216540,Sharjah,2020-10-12,AB de Villiers,Sharjah Cricket Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,runs,82.0,N,NaN,RK Illingworth,K Srinivasan,NaN
805,1216541,Abu Dhabi,2020-10-25,BA Stokes,Sheikh Zayed Stadium,0,Mumbai Indians,Rajasthan Royals,Mumbai Indians,bat,Rajasthan Royals,wickets,8.0,N,NaN,UV Gandhe,VK Sharma,NaN
809,1216545,Abu Dhabi,2020-09-26,Shubman Gill,Sheikh Zayed Stadium,0,Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,7.0,N,NaN,CB Gaffaney,VK Sharma,NaN
813,1237178,Abu Dhabi,2020-11-06,KS Williamson,Sheikh Zayed Stadium,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,field,Sunrisers Hyderabad,wickets,6.0,N,NaN,PR Reiffel,S Ravi,NaN


In [583]:
data_without_neutral['win']=np.where(data_without_neutral['winner'] == data_without_neutral['home_venue'], True, False)
data_without_neutral

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,home_venue,win
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,Royal Challengers Bangalore,False
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,Kings XI Punjab,False
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi Capitals,True
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,Mumbai Indians,False
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,Kolkata Knight Riders,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,1216540,Sharjah,2020-10-12,AB de Villiers,Sharjah Cricket Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,runs,82.0,N,NaN,RK Illingworth,K Srinivasan,NaN,False
805,1216541,Abu Dhabi,2020-10-25,BA Stokes,Sheikh Zayed Stadium,0,Mumbai Indians,Rajasthan Royals,Mumbai Indians,bat,Rajasthan Royals,wickets,8.0,N,NaN,UV Gandhe,VK Sharma,NaN,False
809,1216545,Abu Dhabi,2020-09-26,Shubman Gill,Sheikh Zayed Stadium,0,Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,7.0,N,NaN,CB Gaffaney,VK Sharma,NaN,False
813,1237178,Abu Dhabi,2020-11-06,KS Williamson,Sheikh Zayed Stadium,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,field,Sunrisers Hyderabad,wickets,6.0,N,NaN,PR Reiffel,S Ravi,NaN,False


In [584]:
no_of_wins_homeground=data_without_neutral[data_without_neutral['win']==True]
no_of_wins_homeground_per_team=no_of_wins_homeground.groupby('winner').size().reset_index(name="home_wins_count")
no_of_wins_homeground_per_team=no_of_wins_homeground_per_team.rename(columns={'winner':'team_name'})
no_of_wins_homeground_per_team

,team_name,home_wins_count
0,Chennai Super Kings,40
1,Delhi Capitals,31
2,Gujarat Lions,5
3,Kings XI Punjab,30
4,Kochi Tuskers Kerala,2
5,Kolkata Knight Riders,45
6,Mumbai Indians,53
7,Rajasthan Royals,39
8,Rising Pune Supergiants,11
9,Royal Challengers Bangalore,37


In [585]:
total_home_matches_per_team=data_without_neutral.groupby('home_venue').size().reset_index(name="home_match_count")
total_home_matches_per_team=total_home_matches_per_team.rename(columns={'home_venue':'team_name'})

In [586]:
total_home_matches_per_team

,team_name,home_match_count
0,Chennai Super Kings,57
1,Delhi Capitals,74
2,Gujarat Lions,14
3,Kings XI Punjab,56
4,Kochi Tuskers Kerala,5
5,Kolkata Knight Riders,77
6,Mumbai Indians,101
7,Rajasthan Royals,59
8,Rising Pune Supergiants,51
9,Royal Challengers Bangalore,80


In [587]:
home_match_data=total_home_matches_per_team.merge(no_of_wins_homeground_per_team,left_on='team_name',right_on='team_name', how='left')
home_match_data

,team_name,home_match_count,home_wins_count
0,Chennai Super Kings,57,40
1,Delhi Capitals,74,31
2,Gujarat Lions,14,5
3,Kings XI Punjab,56,30
4,Kochi Tuskers Kerala,5,2
5,Kolkata Knight Riders,77,45
6,Mumbai Indians,101,53
7,Rajasthan Royals,59,39
8,Rising Pune Supergiants,51,11
9,Royal Challengers Bangalore,80,37


In [588]:
home_match_data['%home_win']=np.round((home_match_data['home_wins_count']/home_match_data['home_match_count'])*100,2)
home_match_data

,team_name,home_match_count,home_wins_count,%home_win
0,Chennai Super Kings,57,40,70.18
1,Delhi Capitals,74,31,41.89
2,Gujarat Lions,14,5,35.71
3,Kings XI Punjab,56,30,53.57
4,Kochi Tuskers Kerala,5,2,40.00
5,Kolkata Knight Riders,77,45,58.44
6,Mumbai Indians,101,53,52.48
7,Rajasthan Royals,59,39,66.10
8,Rising Pune Supergiants,51,11,21.57
9,Royal Challengers Bangalore,80,37,46.25


In [589]:
del home_match_data['home_match_count']
del home_match_data['home_wins_count']
home_match_data

,team_name,%home_win
0,Chennai Super Kings,70.18
1,Delhi Capitals,41.89
2,Gujarat Lions,35.71
3,Kings XI Punjab,53.57
4,Kochi Tuskers Kerala,40.00
5,Kolkata Knight Riders,58.44
6,Mumbai Indians,52.48
7,Rajasthan Royals,66.10
8,Rising Pune Supergiants,21.57
9,Royal Challengers Bangalore,46.25


In [590]:
team_df=team_df.merge(home_match_data, left_on='team_name', right_on='team_name', how='left')
team_df

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00
5,Kolkata Knight Riders,95,97,192,99,57,41,1.0,98,57.58,41.41,1.0,51.0,58.44
6,Mumbai Indians,97,106,203,120,56,62,2.0,106,46.67,51.67,1.7,52.2,52.48
7,Rajasthan Royals,70,91,161,81,51,28,2.0,87,62.96,34.57,2.5,54.0,66.10
8,Rising Pune Supergiants,37,39,76,27,14,13,NaN,33,51.85,48.15,NaN,43.4,21.57
9,Royal Challengers Bangalore,108,87,195,91,51,38,2.0,87,56.04,41.76,2.2,44.6,46.25


In [591]:
df['year']=df['date'].map(lambda x:x[0:4])
df

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,year
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,2008
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,2008
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,2008
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,1216547,Dubai,2020-09-28,AB de Villiers,Dubai International Cricket Stadium,0,Royal Challengers Bangalore,Mumbai Indians,Mumbai Indians,field,Royal Challengers Bangalore,tie,NaN,Y,NaN,Nitin Menon,PR Reiffel,2020
812,1237177,Dubai,2020-11-05,JJ Bumrah,Dubai International Cricket Stadium,0,Mumbai Indians,Delhi Capitals,Delhi Capitals,field,Mumbai Indians,runs,57.0,N,NaN,CB Gaffaney,Nitin Menon,2020
813,1237178,Abu Dhabi,2020-11-06,KS Williamson,Sheikh Zayed Stadium,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,field,Sunrisers Hyderabad,wickets,6.0,N,NaN,PR Reiffel,S Ravi,2020
814,1237180,Abu Dhabi,2020-11-08,MP Stoinis,Sheikh Zayed Stadium,0,Delhi Capitals,Sunrisers Hyderabad,Delhi Capitals,bat,Delhi Capitals,runs,17.0,N,NaN,PR Reiffel,S Ravi,2020


In [592]:
year_wise_match_team1=df.groupby(['year','team1']).size().reset_index(name="no_of_match_team1")
year_wise_match_team1=year_wise_match_team1.rename(columns={'team1':'team_name'})
year_wise_match_team1

,year,team_name,no_of_match_team1
0,2008,Chennai Super Kings,9
1,2008,Delhi Capitals,7
2,2008,Kings XI Punjab,7
3,2008,Kolkata Knight Riders,7
4,2008,Mumbai Indians,7
...,...,...,...
102,2020,Kolkata Knight Riders,10
103,2020,Mumbai Indians,9
104,2020,Rajasthan Royals,4
105,2020,Royal Challengers Bangalore,10


In [593]:
year_wise_match_team2=df.groupby(['year','team2']).size().reset_index(name="no_of_match_team2")
year_wise_match_team2=year_wise_match_team2.rename(columns={'team2':'team_name'})
year_wise_match_team2

,year,team_name,no_of_match_team2
0,2008,Chennai Super Kings,7
1,2008,Delhi Capitals,7
2,2008,Kings XI Punjab,8
3,2008,Kolkata Knight Riders,6
4,2008,Mumbai Indians,7
...,...,...,...
102,2020,Kolkata Knight Riders,4
103,2020,Mumbai Indians,7
104,2020,Rajasthan Royals,10
105,2020,Royal Challengers Bangalore,5


In [594]:
year_wise_matches=pd.merge(year_wise_match_team1,year_wise_match_team2,on=['year','team_name'])
year_wise_matches['total_matches']=year_wise_matches['no_of_match_team1']+year_wise_matches['no_of_match_team2']
year_wise_matches

,year,team_name,no_of_match_team1,no_of_match_team2,total_matches
0,2008,Chennai Super Kings,9,7,16
1,2008,Delhi Capitals,7,7,14
2,2008,Kings XI Punjab,7,8,15
3,2008,Kolkata Knight Riders,7,6,13
4,2008,Mumbai Indians,7,7,14
...,...,...,...,...,...
101,2020,Kolkata Knight Riders,10,4,14
102,2020,Mumbai Indians,9,7,16
103,2020,Rajasthan Royals,4,10,14
104,2020,Royal Challengers Bangalore,10,5,15


In [595]:
year_wise_qualified=year_wise_matches[year_wise_matches['total_matches']>14]
year_wise_qualified

,year,team_name,no_of_match_team1,no_of_match_team2,total_matches
0,2008,Chennai Super Kings,9,7,16
2,2008,Kings XI Punjab,7,8,15
5,2008,Rajasthan Royals,7,9,16
9,2009,Delhi Capitals,9,6,15
13,2009,Sunrisers Hyderabad,10,6,16
14,2010,Chennai Super Kings,9,7,16
18,2010,Mumbai Indians,7,9,16
20,2010,Royal Challengers Bangalore,9,7,16
21,2010,Sunrisers Hyderabad,7,9,16
22,2011,Chennai Super Kings,8,8,16


In [596]:
no_of_time_team_qualified=year_wise_qualified.groupby('team_name').size().reset_index(name='toatal_no_of_time_qualified')
no_of_time_team_qualified

,team_name,toatal_no_of_time_qualified
0,Chennai Super Kings,9
1,Delhi Capitals,5
2,Gujarat Lions,1
3,Kings XI Punjab,4
4,Kolkata Knight Riders,7
5,Mumbai Indians,9
6,Rajasthan Royals,4
7,Rising Pune Supergiants,3
8,Royal Challengers Bangalore,7
9,Sunrisers Hyderabad,8


In [597]:
team_df=team_df.merge(no_of_time_team_qualified,left_on='team_name',right_on='team_name',how='left')
team_df

,team_name,no_of_match_as_team1,no_of_match_as_team2,total_no_of_matches_played,no_of_wins,wins_chase,wins_defend,wins_tie,toss_wins,%win_chase,%win_defend,%win_tie,%toss_wins,%home_win,toatal_no_of_time_qualified
0,Chennai Super Kings,94,84,178,106,53,53,NaN,97,50.00,50.00,NaN,54.5,70.18,9.0
1,Delhi Capitals,102,92,194,86,50,34,2.0,100,58.14,39.53,2.3,51.5,41.89,5.0
2,Gujarat Lions,16,14,30,13,12,1,NaN,15,92.31,7.69,NaN,50.0,35.71,1.0
3,Kings XI Punjab,92,98,190,88,45,40,3.0,85,51.14,45.45,3.4,44.7,53.57,4.0
4,Kochi Tuskers Kerala,7,7,14,6,4,2,NaN,8,66.67,33.33,NaN,57.1,40.00,NaN
5,Kolkata Knight Riders,95,97,192,99,57,41,1.0,98,57.58,41.41,1.0,51.0,58.44,7.0
6,Mumbai Indians,97,106,203,120,56,62,2.0,106,46.67,51.67,1.7,52.2,52.48,9.0
7,Rajasthan Royals,70,91,161,81,51,28,2.0,87,62.96,34.57,2.5,54.0,66.10,4.0
8,Rising Pune Supergiants,37,39,76,27,14,13,NaN,33,51.85,48.15,NaN,43.4,21.57,3.0
9,Royal Challengers Bangalore,108,87,195,91,51,38,2.0,87,56.04,41.76,2.2,44.6,46.25,7.0


# Match KPIs

1. Percentage wins by chasing
2. Percentage wins by defending
3. Perceptage of tie matches
4. Percentage won the toss as well as match

In [563]:
#Excluding tie matches
total_matches = df[df['result'] != 'tie']
total_matches["result"].count()

799

In [564]:
run_wins = total_matches[total_matches['result'] == 'runs']
run_wins['result'].count()

364

In [565]:
wicket_wins = total_matches[total_matches['result'] == 'wickets']
wicket_wins['result'].count()

435

In [566]:
#Percentage wins by chasing runs
np.round((run_wins['result'].count()/total_matches["result"].count())*100,1)

45.6

In [567]:
#Percentage wins by defending
np.round((wicket_wins['result'].count()/total_matches["result"].count())*100,1)

54.4

In [568]:
#Perceptage of tie matches
tie_matches = df[df['result'] == 'tie']
np.round(tie_matches['result'].count()/df["result"].count()*100,1)

1.6

In [569]:
#Percentage won the toss as well as match
toss_winner = df[df['toss_winner'] == df['winner']]
np.round(toss_winner['result'].count()/df["result"].count()*100,1)

51.5

In [599]:
mergedTable = data.merge(df, left_on='id',right_on='id',how='left')
mergedTable

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,...,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,year
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,...,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193463,1237181,1,12,5,RR Pant,SS Iyer,NM Coulter-Nile,0,0,0,...,Delhi Capitals,bat,Mumbai Indians,wickets,5.0,N,NaN,CB Gaffaney,Nitin Menon,2020
193464,1237181,1,12,6,RR Pant,SS Iyer,NM Coulter-Nile,1,0,1,...,Delhi Capitals,bat,Mumbai Indians,wickets,5.0,N,NaN,CB Gaffaney,Nitin Menon,2020
193465,1237181,1,13,1,RR Pant,SS Iyer,KH Pandya,0,1,1,...,Delhi Capitals,bat,Mumbai Indians,wickets,5.0,N,NaN,CB Gaffaney,Nitin Menon,2020
193466,1237181,1,13,2,RR Pant,SS Iyer,KH Pandya,1,0,1,...,Delhi Capitals,bat,Mumbai Indians,wickets,5.0,N,NaN,CB Gaffaney,Nitin Menon,2020


In [607]:
year_data = mergedTable.groupby(['year'])['total_runs'].agg(sum).reset_index()
year_data

,year,total_runs
0,2008,17937
1,2009,16320
2,2010,18864
3,2011,21154
4,2012,22453
5,2013,22541
6,2014,18909
7,2015,18332
8,2016,18862
9,2017,18769


In [612]:
year_matches = mergedTable.groupby(['year'])['id'].nunique().reset_index()
year_matches.columns = ['year','matches']
year_matches

,year,matches
0,2008,58
1,2009,57
2,2010,60
3,2011,73
4,2012,74
5,2013,76
6,2014,60
7,2015,59
8,2016,60
9,2017,59


In [613]:
year_data = pd.merge(year_data,year_matches, on='year')
year_data['avg_runs'] = year_data['total_runs']/year_data['matches']
year_data

,year,total_runs,matches,avg_runs
0,2008,17937,58,309.258621
1,2009,16320,57,286.315789
2,2010,18864,60,314.400000
3,2011,21154,73,289.780822
4,2012,22453,74,303.418919
5,2013,22541,76,296.592105
6,2014,18909,60,315.150000
7,2015,18332,59,310.711864
8,2016,18862,60,314.366667
9,2017,18769,59,318.118644
